Importação das Bibliotecas Necessárias

In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Definição do Diretório Base e Criação das Pastas

In [8]:
# Diretório base para as camadas do ETL
base_dir = '/home/savidotti/programmers/credit-risk-analysis/pipeline_etl/etl_camadas'

# Função para criar o diretório se não existir
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

Camada Bronze: Carregamento dos Dados Brutos

In [9]:
# Caminho para o arquivo CSV original
raw_data_path = '/home/savidotti/programmers/credit-risk-analysis/dataset/credit_risk_dataset.csv'

# Carregando os dados brutos
df_bronze = pd.read_csv(raw_data_path)

# Visualizando as primeiras linhas do DataFrame
print("Dados da Camada Bronze:")
print(df_bronze.head())

# Criando o diretório para a camada bronze
bronze_dir = os.path.join(base_dir, 'bronze')
create_dir(bronze_dir)

# Salvando o DataFrame da camada bronze em formato CSV
bronze_file_path = os.path.join(bronze_dir, 'credit_risk_bronze.csv')
df_bronze.to_csv(bronze_file_path, index=False)


Dados da Camada Bronze:
   person_age  person_income person_home_ownership  person_emp_length  \
0          22          59000                  RENT              123.0   
1          21           9600                   OWN                5.0   
2          25           9600              MORTGAGE                1.0   
3          23          65500                  RENT                4.0   
4          24          54400                  RENT                8.0   

  loan_intent loan_grade  loan_amnt  loan_int_rate  loan_status  \
0    PERSONAL          D      35000          16.02            1   
1   EDUCATION          B       1000          11.14            0   
2     MEDICAL          C       5500          12.87            1   
3     MEDICAL          C      35000          15.23            1   
4     MEDICAL          C      35000          14.27            1   

   loan_percent_income cb_person_default_on_file  cb_person_cred_hist_length  
0                 0.59                         Y       

Camada Silver: Limpeza e Transformação Básica

In [10]:
# Criando uma cópia dos dados brutos
df_silver = df_bronze.copy()

# Verificando valores ausentes
print("\nValores ausentes antes do tratamento:")
print(df_silver.isnull().sum())

# Tratamento de valores ausentes (se houver)
if df_silver['person_emp_length'].isnull().sum() > 0:
    df_silver['person_emp_length'].fillna(df_silver['person_emp_length'].median(), inplace=True)

# Convertendo tipos de dados
df_silver['person_age'] = df_silver['person_age'].astype(int)
df_silver['person_income'] = df_silver['person_income'].astype(float)
df_silver['loan_amnt'] = df_silver['loan_amnt'].astype(float)
df_silver['loan_int_rate'] = df_silver['loan_int_rate'].astype(float)
df_silver['loan_status'] = df_silver['loan_status'].astype(int)
df_silver['loan_percent_income'] = df_silver['loan_percent_income'].astype(float)
df_silver['cb_person_cred_hist_length'] = df_silver['cb_person_cred_hist_length'].astype(int)

# Verificando valores ausentes após o tratamento
print("\nValores ausentes após o tratamento:")
print(df_silver.isnull().sum())

# Criando o diretório para a camada silver
silver_dir = os.path.join(base_dir, 'silver')
create_dir(silver_dir)

# Salvando o DataFrame da camada silver em formato Parquet
silver_file_path = os.path.join(silver_dir, 'credit_risk_silver.parquet')
df_silver.to_parquet(silver_file_path, index=False)



Valores ausentes antes do tratamento:
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

Valores ausentes após o tratamento:
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length                0
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64


Camada Gold: Enriquecimento e Preparação para ML

In [14]:
# Criando uma cópia dos dados da camada silver
df_gold = df_silver.copy()

# Engenharia de Características

# Criar uma variável binária para indicar se a pessoa possui casa própria
df_gold['is_home_owner'] = df_gold['person_home_ownership'].apply(lambda x: 1 if x in ['OWN', 'MORTGAGE'] else 0)

# Tratamento de valores nulos em 'person_home_ownership' (se houver)
if df_gold['person_home_ownership'].isnull().sum() > 0:
    df_gold['person_home_ownership'].fillna('OTHER', inplace=True)

# Codificação de variáveis categóricas
categorical_vars = ['loan_intent', 'loan_grade', 'cb_person_default_on_file']

# Usando one-hot encoding para variáveis categóricas
df_gold = pd.get_dummies(df_gold, columns=categorical_vars, drop_first=True)

# Atualizando a lista de features com as novas colunas criadas
features = [
    'person_age',
    'person_income',
    'person_emp_length',
    'loan_amnt',
    'loan_int_rate',
    'loan_percent_income',
    'cb_person_cred_hist_length',
    'is_home_owner'
]

# Adicionando as colunas de variáveis dummies
features += [
    col for col in df_gold.columns 
    if col.startswith('loan_intent_') 
    or col.startswith('loan_grade_') 
    or col.startswith('cb_person_default_on_file_')
]

# Definindo as variáveis X e y
X = df_gold[features]
y = df_gold['loan_status']

# Criando o diretório para a camada gold
gold_dir = os.path.join(base_dir, 'gold')
create_dir(gold_dir)

# Salvando o DataFrame da camada gold em formato Parquet
gold_file_path = os.path.join(gold_dir, 'credit_risk_gold.parquet')
df_gold.to_parquet(gold_file_path, index=False)



NameError: name 'show' is not defined

NameError: name 'x' is not defined

In [ ]:
person_age